# JuliaDB - tutorial

ref. https://www.youtube.com/watch?v=pv5zfIs2lyU, Using table  
ref. https://www.youtube.com/watch?v=eQORf4m_8Hk, Selections in JuliaDB  
ref. https://www.youtube.com/watch?v=tCVgn9m2ajM, Reductions in JuliaDB  
ref. https://www.youtube.com/watch?v=hU7N-EWnC2A, Grouping in JuliaDB to calculate descriptive statistics  
ref. https://www.youtube.com/watch?v=CIBxY7PUjc0, Joining 2 tables  
ref. https://www.youtube.com/watch?v=slL_oj5Dmzc, Importing a csv file in JuliaDB  

ref. https://juliacomputing.github.io/JuliaDB

In [1]:
import Pkg;
Pkg.status()

    Status `~/.julia/environments/v1.2/Project.toml`
  [fbb218c0] BSON v0.2.4
  [6e4b80f9] BenchmarkTools v0.4.3
  [336ed68f] CSV v0.5.14
  [3895d2a7] CUDAapi v1.2.0
  [c5f51814] CUDAdrv v4.0.3
  [be33ccc6] CUDAnative v2.5.4
  [944b1d66] CodecZlib v0.6.0
  [5ae59095] Colors v0.9.6
  [a81c6b42] Compose v0.7.3
  [8f4d0f93] Conda v1.3.0
  [3a865a2d] CuArrays v1.4.4
  [a93c6f00] DataFrames v0.19.4
  [aaf54ef3] DistributedArrays v0.6.4
  [31c24e10] Distributions v0.21.9
  [7876af07] Example v0.5.3
  [587475ba] Flux v0.10.0
  [f6369f11] ForwardDiff v0.10.6
  [c91e804a] Gadfly v1.0.1
  [708ec375] Gumbo v0.5.1
  [cd3eb016] HTTP v0.8.8
  [0e44f5e4] Hwloc v1.0.3
  [7073ff75] IJulia v1.20.2
  [6218d12a] ImageMagick v0.7.5
  [916415d5] Images v0.18.0
  [033835bb] JLD2 v0.1.3
  [682c06a0] JSON v0.21.0
  [18b7da76] JuliaAcademyData v0.1.0 #master (https://github.com/JuliaComputing/JuliaAcademyData.jl)
  [a93385a2] JuliaDB v0.13.0
  [1902f260] Knet v1.3.1
  [7eb4fadd] Match v1.0.2
  [a15396b6] Online

In [2]:
using JuliaDB, Distributions, OnlineStats

┌ Info: Recompiling stale cache file /home/pascal/.julia/compiled/v1.2/Distributions/xILW0.ji for Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1240


## The `table()` function

ref. https://www.youtube.com/watch?v=pv5zfIs2lyU, Using table

In [4]:
ids = collect(1:10)
ages = round.(Int8, rand(Normal(50, 20), 10)); # Normal distribution with μ=50 and σ^2 = 20

In [5]:
length(ids), length(ages)

(10, 10)

In [6]:
typeof(ids), typeof(ages)

(Array{Int64,1}, Array{Int8,1})

In [7]:
# Create table using previously defined
table_1 = table(ids, ages, names = [:ID, :AGE])

Table with 10 rows, 2 columns:
ID  AGE
───────
1   39
2   38
3   54
4   40
5   51
6   77
7   28
8   95
9   70
10  62

In [8]:
subject_3 = table_1[3]

(ID = 3, AGE = 54)

In [9]:
subject_3.AGE

54

In [10]:
table_1[3:5]

Table with 3 rows, 2 columns:
ID  AGE
───────
3   54
4   40
5   51

In [12]:
table_1[[3, 5, 8]]

Table with 3 rows, 2 columns:
ID  AGE
───────
3   54
5   51
8   95

In [20]:
table_2 = table(ids,
    ["I", "I", "II", "III", "III", "II", "I", "III", "II", "I"],
    round.(rand(Normal(16, 4), 10), digits=1), # # Normal distribution with μ=16 and σ^2 = 4, 1 dec place
    round.(rand(Normal(12, 3), 10), digits=1), # Normal distribution with μ=12 and σ^2 = 3, 1 dec place
    names = [:ID, :GROUP, :HB, :WCC], pkey = :ID)

Table with 10 rows, 4 columns:
ID  GROUP  HB    WCC
─────────────────────
1   "I"    14.5  8.1
2   "I"    15.5  13.6
3   "II"   10.3  15.1
4   "III"  22.3  18.0
5   "III"  17.7  12.6
6   "II"   24.2  13.5
7   "I"    15.4  9.5
8   "III"  20.8  4.7
9   "II"   18.3  11.3
10  "I"    14.6  12.5

In [22]:
table_3 = table(Columns(ID = ids, CRP = round.(Int16, rand(Normal(100, 20), 10))), pkey = :ID)

Table with 10 rows, 2 columns:
ID  CRP
───────
1   107
2   99
3   86
4   100
5   72
6   86
7   53
8   99
9   100
10  110

## Selections

ref. https://www.youtube.com/watch?v=eQORf4m_8Hk, Selections in JuliaDB

In [23]:
# create new (sub-)table, using filter function passing a lambda and a table
grouped_by_I = filter(t -> t.GROUP == "I", table_2)

Table with 4 rows, 4 columns:
ID  GROUP  HB    WCC
─────────────────────
1   "I"    14.5  8.1
2   "I"    15.5  13.6
7   "I"    15.4  9.5
10  "I"    14.6  12.5

In [24]:
typeof(grouped_by_I)

IndexedTable{StructArrays.StructArray{NamedTuple{(:ID, :GROUP, :HB, :WCC),Tuple{Int64,String,Float64,Float64}},1,NamedTuple{(:ID, :GROUP, :HB, :WCC),Tuple{Array{Int64,1},Array{String,1},Array{Float64,1},Array{Float64,1}}},Int64}}

In [25]:
grouped_by_II = filter(t -> t.GROUP == "II", table_2)

Table with 3 rows, 4 columns:
ID  GROUP  HB    WCC
─────────────────────
3   "II"   10.3  15.1
6   "II"   24.2  13.5
9   "II"   18.3  11.3

In [26]:
grouped_by_III = filter(t -> t.GROUP == "III", table_2)

Table with 3 rows, 4 columns:
ID  GROUP  HB    WCC
─────────────────────
4   "III"  22.3  18.0
5   "III"  17.7  12.6
8   "III"  20.8  4.7

In [27]:
high_WCC = filter(t -> t.WCC > 12, table_2)

Table with 6 rows, 4 columns:
ID  GROUP  HB    WCC
─────────────────────
2   "I"    15.5  13.6
3   "II"   10.3  15.1
4   "III"  22.3  18.0
5   "III"  17.7  12.6
6   "II"   24.2  13.5
10  "I"    14.6  12.5

## Reductions

ref. https://www.youtube.com/watch?v=eQORf4m_8Hk, Selections in JuliaDB 

In [28]:
table_2

Table with 10 rows, 4 columns:
ID  GROUP  HB    WCC
─────────────────────
1   "I"    14.5  8.1
2   "I"    15.5  13.6
3   "II"   10.3  15.1
4   "III"  22.3  18.0
5   "III"  17.7  12.6
6   "II"   24.2  13.5
7   "I"    15.4  9.5
8   "III"  20.8  4.7
9   "II"   18.3  11.3
10  "I"    14.6  12.5

In [29]:
# sum on column :WCC
reduce(+, table_2, select = :WCC)

118.89999999999999

In [33]:
# min and max values on column :WCC
# NOTE: we pass a tuple to reduce

reduce((min, max), table_2, select = :WCC)

(min = 4.7, max = 18.0)

In [41]:
reduce((Mean(), Variance()), table_2, select = :WCC) # using function from OnlineStats package

(Mean = Mean: n=10 | value=11.89, Variance = Variance: n=10 | value=14.0166)

## Grouping

ref. https://www.youtube.com/watch?v=hU7N-EWnC2A, Grouping in JuliaDB to calculate descriptive statistics  

In [42]:
# Calculate the mean of HB per group
groupreduce(Mean(), table_2, :GROUP, select = :HB)

Table with 3 rows, 2 columns:
GROUP  Mean
────────────────────────────────
"I"    Mean: n=4 | value=15.0
"II"   Mean: n=3 | value=17.6
"III"  Mean: n=3 | value=20.2667

In [44]:
groupreduce(Variance(), table_2, :GROUP, select = :HB)

Table with 3 rows, 2 columns:
GROUP  Variance
─────────────────────────────────────
"I"    Variance: n=4 | value=0.273333
"II"   Variance: n=3 | value=48.67
"III"  Variance: n=3 | value=5.50333

In [45]:
groupby((mean, median, std, var, quantile), table_2, :GROUP, select = :HB)

┌ Warning: `quantile(v::AbstractArray{<:Real})` is deprecated, use `quantile(v, [0.0, 0.25, 0.5, 0.75, 1.0])` instead.
│   caller = _apply at columns.jl:582 [inlined]
└ @ Core /home/pascal/.julia/packages/IndexedTables/5U0Ap/src/columns.jl:582


Table with 3 rows, 6 columns:
GROUP  mean     median  std       var       quantile
──────────────────────────────────────────────────────────────────────────────
"I"    15.0     15.0    0.522813  0.273333  [14.5, 14.575, 15.0, 15.425, 15.5]
"II"   17.6     18.3    6.97639   48.67     [10.3, 14.3, 18.3, 21.25, 24.2]
"III"  20.2667  20.8    2.34592   5.50333   [17.7, 19.25, 20.8, 21.55, 22.3]

## Joining 2 tables

ref. https://www.youtube.com/watch?v=CIBxY7PUjc0, Joining 2 tables  

In [46]:
left_table = table(ids, ages, names = [:ID, :AGE], pkey = :ID)

Table with 10 rows, 2 columns:
ID  AGE
───────
1   39
2   38
3   54
4   40
5   51
6   77
7   28
8   95
9   70
10  62

In [48]:
right_table = table(collect(1:15), round.(Int16, rand(Normal(100, 20), 15)), names = [:ID, :GROUP], pkey = :ID)

Table with 15 rows, 2 columns:
ID  GROUP
─────────
1   132
2   99
3   107
4   101
5   140
6   92
7   130
8   128
9   89
10  98
11  89
12  105
13  103
14  122
15  81

In [49]:
join(left_table, right_table) # inner-join the 2 tables on ID

Table with 10 rows, 3 columns:
ID  AGE  GROUP
──────────────
1   39   132
2   38   99
3   54   107
4   40   101
5   51   140
6   77   92
7   28   130
8   95   128
9   70   89
10  62   98

In [50]:
join(right_table, left_table) # inner-join the 2 tables on ID

Table with 10 rows, 3 columns:
ID  GROUP  AGE
──────────────
1   132    39
2   99     38
3   107    54
4   101    40
5   140    51
6   92     77
7   130    28
8   128    95
9   89     70
10  98     62

In [51]:
join(left_table, right_table, how = :left) # inner-join the 2 tables on ID, from left table

Table with 10 rows, 3 columns:
ID  AGE  GROUP
──────────────
1   39   132
2   38   99
3   54   107
4   40   101
5   51   140
6   77   92
7   28   130
8   95   128
9   70   89
10  62   98

In [52]:
join(left_table, right_table, how = :outer) # outer-join left and right table on ID 

Table with 15 rows, 3 columns:
ID  AGE      GROUP
──────────────────
1   39       132
2   38       99
3   54       107
4   40       101
5   51       140
6   77       92
7   28       130
8   95       128
9   70       89
10  62       98
11  missing  89
12  missing  105
13  missing  103
14  missing  122
15  missing  81

In [53]:
join(right_table, left_table, how = :anti) # only tuple that have no id in right table

Table with 5 rows, 2 columns:
ID  GROUP
─────────
11  89
12  105
13  103
14  122
15  81

## Loading a CSV file

ref. https://www.youtube.com/watch?v=slL_oj5Dmzc, Importing a csv file in JuliaDB  

In [71]:
# csv file is in the current directory
db = loadtable("Data.csv", header_exists = true)

Table with 506 rows, 14 columns:
Columns:
#   colname  type
────────────────────
1   506      Float64
2   13       Float64
3   CRIM     Float64
4   ZN       Int64
5   INDUS    Float64
6   CHAS     Float64
7   NOX      Float64
8   RM       Float64
9   AGE      Int64
10  DIS      Int64
11  RAD      Float64
12  TAX      Float64
13  PTRATIO  Float64
14  B        Float64

In [70]:
db[1]

(506 = 0.00632, 13 = 18.0, CRIM = 2.31, ZN = 0, INDUS = 0.538, CHAS = 6.575, NOX = 65.2, RM = 4.09, AGE = 1, DIS = 296, RAD = 15.3, TAX = 396.9, PTRATIO = 4.98, B = 24.0)

In [78]:
t = groupby((mean, median, std, var, quantile), db, 13, select = :CRIM)

Table with 455 rows, 6 columns:
Columns:
#  colname   type
─────────────────────────────
1  PTRATIO   Float64
2  mean      Float64
3  median    Float64
4  std       Float64
5  var       Float64
6  quantile  Array{Float64,1}

In [81]:
length(t)

455

In [79]:
t[2]

(PTRATIO = 1.92, mean = 19.58, median = 19.58, std = NaN, var = NaN, quantile = [19.58, 19.58, 19.58, 19.58, 19.58])

In [80]:
t[1]

(PTRATIO = 1.73, mean = 19.58, median = 19.58, std = NaN, var = NaN, quantile = [19.58, 19.58, 19.58, 19.58, 19.58])

In [82]:
t[1:10]

Table with 10 rows, 6 columns:
PTRATIO  mean   median  std  var  quantile
─────────────────────────────────────────────────────────────────────
1.73     19.58  19.58   NaN  NaN  [19.58, 19.58, 19.58, 19.58, 19.58]
1.92     19.58  19.58   NaN  NaN  [19.58, 19.58, 19.58, 19.58, 19.58]
1.98     2.95   2.95    NaN  NaN  [2.95, 2.95, 2.95, 2.95, 2.95]
2.47     6.2    6.2     NaN  NaN  [6.2, 6.2, 6.2, 6.2, 6.2]
2.87     3.44   3.44    NaN  NaN  [3.44, 3.44, 3.44, 3.44, 3.44]
2.88     2.68   2.68    NaN  NaN  [2.68, 2.68, 2.68, 2.68, 2.68]
2.94     2.18   2.18    NaN  NaN  [2.18, 2.18, 2.18, 2.18, 2.18]
2.96     18.1   18.1    NaN  NaN  [18.1, 18.1, 18.1, 18.1, 18.1]
2.97     0.46   0.46    NaN  NaN  [0.46, 0.46, 0.46, 0.46, 0.46]
2.98     6.41   6.41    NaN  NaN  [6.41, 6.41, 6.41, 6.41, 6.41]

In [93]:
n_t = filter(t -> t.CRIM > 2., db)

Table with 486 rows, 14 columns:
Columns:
#   colname  type
────────────────────
1   506      Float64
2   13       Float64
3   CRIM     Float64
4   ZN       Int64
5   INDUS    Float64
6   CHAS     Float64
7   NOX      Float64
8   RM       Float64
9   AGE      Int64
10  DIS      Int64
11  RAD      Float64
12  TAX      Float64
13  PTRATIO  Float64
14  B        Float64

In [96]:
length(n_t), typeof(n_t)

(486, IndexedTable{StructArrays.StructArray{NamedTuple{(Symbol("506"), Symbol("13"), :CRIM, :ZN, :INDUS, :CHAS, :NOX, :RM, :AGE, :DIS, :RAD, :TAX, :PTRATIO, :B),Tuple{Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,Int64,Int64,Float64,Float64,Float64,Float64}},1,NamedTuple{(Symbol("506"), Symbol("13"), :CRIM, :ZN, :INDUS, :CHAS, :NOX, :RM, :AGE, :DIS, :RAD, :TAX, :PTRATIO, :B),Tuple{Array{Float64,1},Array{Float64,1},Array{Float64,1},Array{Int64,1},Array{Float64,1},Array{Float64,1},Array{Float64,1},Array{Float64,1},Array{Int64,1},Array{Int64,1},Array{Float64,1},Array{Float64,1},Array{Float64,1},Array{Float64,1}}},Int64}})

In [95]:
n_t[1:10]

Table with 10 rows, 14 columns:
Columns:
#   colname  type
────────────────────
1   506      Float64
2   13       Float64
3   CRIM     Float64
4   ZN       Int64
5   INDUS    Float64
6   CHAS     Float64
7   NOX      Float64
8   RM       Float64
9   AGE      Int64
10  DIS      Int64
11  RAD      Float64
12  TAX      Float64
13  PTRATIO  Float64
14  B        Float64

In [97]:
t = table([1,1,2,2], [1,2,1,2], [1,2,3,4], names=[:a,:b,:c], pkey = (:a, :b))

Table with 4 rows, 3 columns:
a  b  c
───────
1  1  1
1  2  2
2  1  3
2  2  4

In [98]:
rows(t)

4-element StructArray(::Array{Int64,1}, ::Array{Int64,1}, ::Array{Int64,1}) with eltype NamedTuple{(:a, :b, :c),Tuple{Int64,Int64,Int64}}:
 (a = 1, b = 1, c = 1)
 (a = 1, b = 2, c = 2)
 (a = 2, b = 1, c = 3)
 (a = 2, b = 2, c = 4)

In [99]:
n_t = merge(t, 
        table([5,5,7], [3,6,3], [10,20,30], names=[:a,:b,:c], pkey = (:a, :b)))

Table with 7 rows, 3 columns:
a  b  c
────────
1  1  1
1  2  2
2  1  3
2  2  4
5  3  10
5  6  20
7  3  30